In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cars = os.path.join("..","electric_cars","vehicle-fuel-type-count-by-zip-code.csv")
geocodes_raw = os.path.join("..","electric_cars","fips_zip_x.csv")
cars_data = pd.read_csv(cars)
geocodes = pd.read_csv(geocodes_raw)
cars_data

,Date,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
0,10/1/2018,90000,2006,Gasoline,OTHER/UNK,Light,1
1,10/1/2018,90000,2014,Gasoline,OTHER/UNK,Light,1
2,10/1/2018,90000,2016,Gasoline,OTHER/UNK,Light,1
3,10/1/2018,90000,2017,Gasoline,OTHER/UNK,Light,1
4,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,55
...,...,...,...,...,...,...,...
584333,10/1/2018,99999,2008,Gasoline,OTHER/UNK,Light,1
584334,10/1/2018,99999,2012,Flex-Fuel,OTHER/UNK,Light,1
584335,10/1/2018,99999,2013,Gasoline,OTHER/UNK,Light,2
584336,10/1/2018,99999,2015,Gasoline,OTHER/UNK,Light,1


In [3]:
print(len(cars_data))
cars_data['Vehicles'].sum()

584338


30411713

In [4]:
len(geocodes)

1934

In [5]:
#getting the number of zips per county, by grouping by zip and then counting number of counties
geocodes = geocodes.rename(columns = {'ZCTA5': 'Zip Code'})
n_zip_county = geocodes.groupby('Zip Code').count().sort_values(['COUNTY'],ascending = False).reset_index()

#creating weights by doing 1/n
n_zip_county['Weights'] = 1/n_zip_county['COUNTY']
n_zip_county

,Zip Code,Unnamed: 0,STATE,COUNTY,Weights
0,93252,4,4,4,0.250000
1,95329,3,3,3,0.333333
2,95230,3,3,3,0.333333
3,93461,3,3,3,0.333333
4,95960,3,3,3,0.333333
...,...,...,...,...,...
1764,92647,1,1,1,1.000000
1765,92646,1,1,1,1.000000
1766,92637,1,1,1,1.000000
1767,92630,1,1,1,1.000000


In [6]:
#renaming zip code for merge
cars_data =cars_data.rename(columns={'Zip Code': 'Zip Code'})

#reducing zip codes to just zip code and weights
n_zip_county = n_zip_county[['Zip Code','Weights']].sort_values(['Zip Code'])
n_zip_county

,Zip Code,Weights
916,89010,1.0
142,89019,0.5
1209,89060,1.0
1208,89061,1.0
1207,89439,1.0
...,...,...
634,96148,1.0
633,96150,1.0
632,96155,1.0
144,96161,0.5


In [7]:
#mapping n_zips back into geocodes
geocodes_weights = pd.merge(geocodes,n_zip_county,
                            on = 'Zip Code',how = 'inner')
geocodes_weights

,Unnamed: 0,Zip Code,STATE,COUNTY,Weights
0,40738,89010,6,51,1.0
1,40748,89019,6,27,0.5
2,40749,89019,6,71,0.5
3,40774,89060,6,27,1.0
4,40776,89061,6,27,1.0
...,...,...,...,...,...
1929,42860,96150,6,17,1.0
1930,42861,96155,6,17,1.0
1931,42862,96161,6,57,0.5
1932,42863,96161,6,61,0.5


In [8]:
#merging weights/number of car regristrations by zip code into fips code
cars_merge_county = pd.merge(cars_data, geocodes_weights, how="inner", on=["Zip Code"])
cars_merge_county.sort_values(['Weights'])

,Date,Zip Code,Model Year,Fuel,Make,Duty,Vehicles,Unnamed: 0,STATE,COUNTY,Weights
298101,10/1/2018,93252,2007,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,4,41696,6,83,0.25
298290,10/1/2018,93252,2017,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,1,41697,6,111,0.25
298289,10/1/2018,93252,2017,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,1,41696,6,83,0.25
298288,10/1/2018,93252,2017,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,1,41695,6,79,0.25
298287,10/1/2018,93252,2017,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,1,41694,6,29,0.25
...,...,...,...,...,...,...,...,...,...,...,...
204392,10/1/2018,92345,2016,Gasoline,CHRYSLER,Light,38,41448,6,71,1.00
204393,10/1/2018,92345,2016,Gasoline,DODGE,Light,86,41448,6,71,1.00
204394,10/1/2018,92345,2016,Gasoline,FORD,Heavy,27,41448,6,71,1.00
204388,10/1/2018,92345,2016,Gasoline,BMW,Light,22,41448,6,71,1.00


In [9]:
#multiplying the weights by number of vehicles
cars_merge_county['Adj Veh'] = cars_merge_county['Vehicles']*cars_merge_county['Weights']

cars_merge_county.sort_values(['Weights']).sort_values('Zip Code').head(10)

,Date,Zip Code,Model Year,Fuel,Make,Duty,Vehicles,Unnamed: 0,STATE,COUNTY,Weights,Adj Veh
74,10/1/2018,90001,2008,Gasoline,FORD,Light,76,40937,6,37,1.0,76.0
113,10/1/2018,90001,2009,Gasoline,SCION,Light,22,40937,6,37,1.0,22.0
112,10/1/2018,90001,2009,Gasoline,OTHER/UNK,Light,55,40937,6,37,1.0,55.0
111,10/1/2018,90001,2009,Gasoline,OTHER/UNK,Heavy,2,40937,6,37,1.0,2.0
110,10/1/2018,90001,2009,Gasoline,NISSAN,Light,119,40937,6,37,1.0,119.0
109,10/1/2018,90001,2009,Gasoline,MERCEDES-BENZ,Light,11,40937,6,37,1.0,11.0
108,10/1/2018,90001,2009,Gasoline,LEXUS,Light,13,40937,6,37,1.0,13.0
107,10/1/2018,90001,2009,Gasoline,KIA,Light,18,40937,6,37,1.0,18.0
106,10/1/2018,90001,2009,Gasoline,INFINITI,Light,17,40937,6,37,1.0,17.0
105,10/1/2018,90001,2009,Gasoline,HYUNDAI,Light,16,40937,6,37,1.0,16.0


In [10]:
sum_cars = cars_merge_county[['Vehicles','Adj Veh','Fuel','COUNTY']].groupby(['Fuel','COUNTY']).sum().rename(columns = {'Vehicles':'Veh','Adj Veh': 'N Adj Veh'}).reset_index()
sum_cars.sort_values('COUNTY')

,Fuel,COUNTY,Veh,N Adj Veh
0,Battery Electric,1,19491,17754.000000
57,Diesel and Diesel Hybrid,1,39748,34839.000000
321,Natural Gas,1,1248,1188.000000
378,Other,1,209,197.500000
289,Hydrogen Fuel Cell,1,321,298.500000
...,...,...,...,...
230,Gasoline,115,71396,46735.333333
377,Natural Gas,115,20,12.500000
172,Flex-Fuel,115,4572,3015.833333
114,Diesel and Diesel Hybrid,115,6567,4063.000000


In [11]:
cars_by_county = cars_merge_county[['Vehicles','Adj Veh','COUNTY']].groupby('COUNTY').sum().rename(columns = {'Vehicles':'tot Veh count','Adj Veh':'tot Adj Veh count'})
len(cars_by_county)

58

In [12]:
cars_percentage = pd.merge(sum_cars, cars_by_county,
                           how = 'left', on = 'COUNTY')
cars_percentage['percentage unadjusted'] = cars_percentage['Veh']/cars_percentage['tot Veh count']
cars_percentage['percentage adjusted'] = cars_percentage['N Adj Veh']/cars_percentage['tot Adj Veh count']


cars_percentage = cars_percentage[['Fuel','COUNTY','percentage unadjusted','percentage adjusted']]
cars_percentage.sort_values('COUNTY').set_index('COUNTY')

,Fuel,percentage unadjusted,percentage adjusted
COUNTY,,,
1,Battery Electric,0.015331,0.015339
1,Diesel and Diesel Hybrid,0.031264,0.030100
1,Natural Gas,0.000982,0.001026
1,Other,0.000164,0.000171
1,Hydrogen Fuel Cell,0.000252,0.000258
...,...,...,...
115,Gasoline,0.846656,0.849224
115,Natural Gas,0.000237,0.000227
115,Flex-Fuel,0.054218,0.054800


In [39]:

#manually transposing percentages to type percentage
batt_elect = cars_percentage.loc[cars_percentage['Fuel'] == 'Battery Electric']
batt_elect = batt_elect.rename(columns = {'percentage unadjusted': 'Battery Electric % Unadj', 'percentage adjusted': 'Battery Electric % adj'}).drop(columns = 'Fuel')


hybrid = cars_percentage.loc[cars_percentage['Fuel'] == 'Hybrid Gasoline']
hybrid = hybrid.rename(columns = {'percentage unadjusted': 'Hybrid Gasoline % Unadj', 'percentage adjusted': 'Hybrid Gasoline % adj'}).drop(columns = 'Fuel')

plug = cars_percentage.loc[cars_percentage['Fuel'] == 'Battery Electric']
plug = plug.rename(columns = {'percentage unadjusted': 'Plug Electric % Unadj', 'percentage adjusted': 'Plug Electric % adj'}).drop(columns = 'Fuel')


#merging back together on county code
bat_hy = pd.merge(batt_elect, hybrid,
                 how = 'right', on ='COUNTY')
final = pd.merge(bat_hy, plug,
                how = 'left', on = 'COUNTY')
final.head()
final['% Unadj Low Emission'] = final['Battery Electric % Unadj'] + final['Hybrid Gasoline % Unadj'] + final['Plug Electric % Unadj']
final['% Adj Low Emission'] = final['Battery Electric % adj'] + final['Hybrid Gasoline % adj'] + final['Plug Electric % adj']

final.head()
final.sort_values('% Adj Low Emission', ascending = False).head(10)

,COUNTY,Battery Electric % Unadj,Battery Electric % adj,Hybrid Gasoline % Unadj,Hybrid Gasoline % adj,Plug Electric % Unadj,Plug Electric % adj,% Unadj Low Emission,% Adj Low Emission
37,75,0.014100,0.014100,0.074846,0.074846,0.014100,0.014100,0.103046,0.103046
20,41,0.016583,0.016946,0.065715,0.066633,0.016583,0.016946,0.098882,0.100525
42,85,0.020393,0.021234,0.047124,0.048145,0.020393,0.021234,0.087911,0.090614
40,81,0.015775,0.015431,0.054039,0.053847,0.015775,0.015431,0.085589,0.084710
0,1,0.015331,0.015339,0.052245,0.051540,0.015331,0.015339,0.082906,0.082218
43,87,0.009033,0.010191,0.046107,0.051759,0.009033,0.010191,0.064173,0.072141
6,13,0.010692,0.010216,0.047636,0.046194,0.010692,0.010216,0.069020,0.066626
47,97,0.008041,0.007979,0.043262,0.043207,0.008041,0.007979,0.059345,0.059166
29,59,0.009736,0.009840,0.035108,0.035163,0.009736,0.009840,0.054581,0.054843
18,37,0.007003,0.006974,0.038594,0.038631,0.007003,0.006974,0.052599,0.052579
